In [2]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/customer-orders.csv


--2023-07-05 00:55:42--  https://s3-geospatial.s3-us-west-2.amazonaws.com/customer-orders.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.248.226, 52.218.247.2, 52.218.182.89, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.248.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146855 (143K) [text/csv]
Saving to: ‘customer-orders.csv.2’

customer-orders.csv 100%[===================>] 143.41K   434KB/s    in 0.3s    

2023-07-05 00:55:43 (434 KB/s) - ‘customer-orders.csv.2’ saved [146855/146855]



In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master('local[*]')\
    .appName('pyspark_22')\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/05 01:18:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/05 01:18:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
my_df = spark.read.csv('customer-orders.csv')

In [6]:
from pyspark.sql import functions as func
from pyspark.sql.types import StructType,StructField,StringType, FloatType

schema = StructType([\
    StructField("cust_id",StringType(),True),\
    StructField("item_id",StringType(),True),\
    StructField("amount_spent",FloatType(),True)])

my_df = spark.read.schema(schema).csv('customer-orders.csv')


In [7]:
my_df.printSchema()

root
 |-- cust_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- amount_spent: float (nullable = true)



In [8]:
df_ca = my_df.groupby('cust_id').sum('amount_spent')

In [12]:
df_ca = my_df.groupby('cust_id').sum('amount_spent').withColumnRenamed("sum(amount_spent)","sum")

In [20]:
df_ca = my_df.groupby('cust_id').agg(func.sum('amount_spent').alias('sum'))

In [21]:
df_ca.show(5)

+-------+-----------------+
|cust_id|              sum|
+-------+-----------------+
|     51|4975.219970226288|
|      7|4755.070008277893|
|     15|5413.510010659695|
|     54|6065.390002984554|
|     11|5152.289969373494|
+-------+-----------------+
only showing top 5 rows



In [24]:
df_da = my_df.groupby('cust_id')\
    .agg(
    func.sum('amount_spent').alias('sum'),
    func.max('amount_spent').alias('max'),
    func.min('amount_spent').alias('min')
)

In [26]:
my_df.createOrReplaceTempView('customer_order')

In [28]:
my_sql = spark.sql("""
SELECT cust_id, SUM(amount_spent) sum, MAX(amount_spent) max, AVG(amount_spent) avg
FROM customer_order
GROUP BY cust_id
""")

In [31]:
spark.catalog.listTables()

[Table(name='customer_order', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [1]:
print('heeo')

heeo


In [15]:
import os

print(os.environ["JAVA_HOME"])

/Users/giho.han/Library/Java/JavaVirtualMachines/corretto-17.0.7/Contents/Home
